# Optimizando funciones de una variable: minimización de costos

Original en inglés: [Optimizing Functions of One Variable: Cost Minimization](https://github.com/Ryota-Kawamura/Mathematics-for-Machine-Learning-and-Data-Science-Specialization/blob/main/Course-2/Week-1/C2_W1_Assignment.ipynb)


En esta tarea, usted resolverá un problema de optimización simple para una función de una variable. Dado un conjunto de datos de precios históricos de un producto de dos proveedores, su tarea es identificar qué proporción del producto debería comprar a cada uno de los proveedores para realizar la mejor inversión posible en el futuro. Al formular el problema matemáticamente, construirá una función objetivo para minimizar, evaluará su mínimo e investigará cómo se relaciona su derivada con el resultado.

# Tabla de contenidos

## 1 - Planteamiento del problema de optimización

### 1.1 - Descripción del problema

Una empresa tiene como objetivo minimizar los costos de producción de algunos bienes. Durante el proceso de producción, se utiliza un producto esencial $P$, que puede ser suministrado por uno de dos socios: el proveedor $A$ y el proveedor $B$. Sus consultores solicitaron los precios históricos del producto $P$ a los proveedores $A$ y $B$, que se proporcionaron como promedios mensuales para el período de febrero de 2018 a marzo de 2020.

Al preparar el presupuesto de la empresa para el próximo período de doce meses, su plan es comprar la misma cantidad de producto $P$ mensualmente. Al elegir el proveedor, notó que hubo algunos períodos en el pasado en los que sería más rentable utilizar el proveedor $A$ (los precios del producto $P$ eran más bajos) y otros períodos en los que trabajar con el proveedor $B$. Para el modelo de presupuesto, puede establecer un porcentaje de los bienes que se comprarán al proveedor $A$ (por ejemplo, 60%) y la parte restante al proveedor $B$ (por ejemplo, 40%), pero esta división debe mantenerse constante durante todo el período de doce meses. El presupuesto se utilizará en la preparación de las negociaciones del contrato con ambos proveedores.

En base a los precios históricos, ¿existe un porcentaje en particular que será más rentable suministrar a la empresa $A$ y el resto a la empresa $B$? ¿O tal vez no importe y se pueda trabajar solo con uno de los proveedores?

### 1.2 - Formulación matemática del problema

Denominamos los precios del producto $P$ de la Empresa $A$ y la Empresa $B$ como $p_A$ (en dólares norteamericanos o USD) y $p_B$ (en USD) respectivamente, y el volumen del producto a suministrar por mes como $n$ (unidades), el costo total en USD es:

\begin{equation}
f\left(\omega\right) = p_A \omega \,n+ p_B \left(1 - \omega\right) n
\end{equation}

Obviamente, no conoces los precios futuros $p_A$ y $p_B$, solo los valores históricos (precios $\{p_A^1, \cdots, p_A^k\}$ y $\{p_B^1, \cdots, p_B^k\}$ para $k$ meses). E históricamente hubo varios períodos de tiempo en los que era mejor tomar $\omega = 1$ ($p_A^i < p_B^i$) o $\omega = 0$ ($p_A^i >p_B^i$). ¿Es posible ahora elegir algún valor de $\omega$ que proporcione alguna evidencia de costos mínimos en el futuro?

### 1.3 - Enfoque de solución

Este es un problema estándar de **gestión de cartera** (inversión) muy conocido en estadística, en el que, en función de los precios históricos, es necesario tomar decisiones de inversión para maximizar las ganancias (minimizar los costos). Dado que en este curso no se han abordado cálculos estadísticas, no es necesario que comprenda los detalles sobre la función $\mathcal{L}\left(\omega\right)$ (llamada **función de pérdida**) para minimizar, que se explica en el siguiente párrafo.

El enfoque consiste en calcular $f\left(\omega\right)$ para cada uno de los precios históricos $p_A^i$ y $p_B^i$, $f^i\left(\omega\right)=p_A^i \omega+ p_B^i \left(1 - \omega\right)$. Luego, toma un promedio de esos valores, $\overline{f\left (\omega\right)}=\text{mean}\left(f^i\left(\omega\right)\right) = \frac{1}{k}\sum_{i=1}^{k}f^i\left(\omega\right)$ y busca el valor de $\omega$ que haga que $f^i\left(\omega\right)$ sea lo más "estable" posible, variando lo menos posible del promedio $\overline{f\left (\omega\right)}$. Esto significa que querrás minimizar la suma de las diferencias $\left(f^i \left(\omega\right) - \overline{f\left (\omega\right)}\right)$. Como las diferencias pueden ser negativas o positivas, un enfoque común es tomar los cuadrados de éstas y sacar un promedio de los cuadrados:

\begin{equation}
\mathcal{L}\left(\omega\right) = \frac{1}{k}\sum_{i=1}^{k}\left(f^i \left(\omega\right) -  \overline{f\left (\omega\right)}\right)^2\tag{2}
\end{equation}

En estadística, $\mathcal{L}\left(\omega\right)$ se denomina varianza de $\{f^1 \left(\omega\right), \cdots , f^k \left(\omega\right)\}$. El objetivo es minimizar la varianza $\mathcal{L}\left(\omega\right)$, donde $\omega\in\left[0, 1\right]$. Nuevamente, no se preocupe si no comprende profundamente por qué se eligió en particular esta función $\mathcal{L}\left(\omega\right)$. Se podría pensar que es lógico minimizar un promedio $\overline{f\left (\omega\right)}$, pero la teoría de [gestión de riesgos](https://www.thebalancemoney.com/minimum-variance-portfolio-overview-4155796#:~:text=A%20minimum%20variance%20portfolio%20is,other%20out%20when%20held%20together.) establece que en este problema es necesario optimizar la varianza.

## 2 - Optimizando una función de una variable en Python

### 2.1 - Paquetes

Importemos todos los paquetes necesarios. Deberá importar la biblioteca "pandas", el paquete que se utiliza habitualmente para la manipulación y el análisis de datos.

In [ ]:
# A function to perform automatic differentiation.
from jax import grad
# A wrapped version of NumPy to use JAX primitives.
import jax.numpy as np
# A library for programmatic plot generation.
import matplotlib.pyplot as plt
# A library for data manipulation and analysis.
import pandas as pd

# A magic command to make output of plotting commands displayed inline within the Jupyter notebook.
%matplotlib inline

### 2.2 - Abrir y analizar el conjunto de datos

Los precios históricos de los proveedores A y B se guardan en el archivo `data/prices.csv`. Para abrirlo, puede utilizar la función `read_csv` de `pandas`. Este ejemplo es muy sencillo, no es necesario utilizar ningún otro parámetro.

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/Ryota-Kawamura/Mathematics-for-Machine-Learning-and-Data-Science-Specialization/refs/heads/main/Course-2/Week-1/data/prices.csv')

Los datos ahora se guardan en la variable `df` como un **DataFrame**, que es el objeto `pandas` más comúnmente utilizado. Es una estructura de datos etiquetada bidimensional con columnas de tipos potencialmente diferentes. Puedes pensar en ella como una tabla o una hoja de cálculo. Puedes encontrar la documentación completa [aquí](https://pandas.pydata.org/).

Ver los datos con una función `print` estándar:

In [ ]:
print(df)

Para imprimir una lista de los nombres de las columnas, utilice el atributo `columns` del DataFrame:

In [ ]:
print(df.columns)

Al revisar la tabla mostrada y los nombres de las columnas, puede concluir que se proporcionan los precios mensuales (en USD) y que solo necesita los datos de las columnas `price_supplier_a_dollars_per_item` y `price_supplier_b_dollars_per_item`. En la vida real, los conjuntos de datos son significativamente más grandes y requieren una revisión y limpieza adecuadas antes de la inyección en los modelos. Pero este no es el objetivo de este curso.

Para acceder a los valores de una columna del DataFrame, puede usar el nombre de la columna como atributo. Por ejemplo, el siguiente código generará la columna `date` del DataFrame `df`:

In [ ]:
df.date

### Ejercicio 1

Cargue los precios históricos del proveedor A y del proveedor B en las variables `prices_A` y `prices_B`, respectivamente. Convierta los valores de precios en matrices `NumPy` con elementos de tipo `float32` mediante la función `np.array`.

**Pista**

- Los precios correspondientes se encuentran en el DataFrame `df`, columnas `price_supplier_a_dollars_per_item` y `price_supplier_b_dollars_per_item`.

- La conversión a la matriz `NumPy` se puede realizar con la función `np.array`.

In [ ]:
### START CODE HERE ### (~ 4 lines of code)
prices_A = df.price_supplier_a_dollars_per_item
prices_B = df.price_supplier_b_dollars_per_item
prices_A = np.array(prices_A).astype('float32')
prices_B = np.array(prices_B).astype('float32')
### END CODE HERE ###

In [ ]:
# Print some elements and mean values of the prices_A and prices_B arrays.
print("Some prices of supplier A:", prices_A[0:5])
print("Some prices of supplier B:", prices_B[0:5])
print("Average of the prices, supplier A:", np.mean(prices_A))
print("Average of the prices, supplier B:", np.mean(prices_B))

Los precios promedio de ambos proveedores son similares, pero si vemos las gráficas de los precios históricos, veremos que hubo períodos en los que los precios fueron más bajos para el proveedor $A$ y viceversa.

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
plt.plot(prices_A, 'g', label="Supplier A")
plt.plot(prices_B, 'b', label="Supplier B")
plt.legend()

plt.show()

Con base en los datos históricos, ¿puede decir con qué proveedor será más rentable trabajar? Como se explicó en la sección [1.3](#1.3), debe encontrar un $\omega \in \left[0, 1\right]$ que minimice la función $(2)$.

### 2.3 - Construyendo la función $\mathcal{L}$ para optimizar y encontrar su punto

### Ejercicio 2

Crea la función de `Python` llamada `f_of_omega`, correspondiente a $f^i\left(\omega\right)=p_A^i \omega+ p_B^i \left(1 - \omega\right)$. Los precios $\{p_A^1, \cdots, p_A^k\}$ y $\{p_B^1, \cdots, p_B^k\}$ se guardan en las matrices `prices_A` y `prices_B`. Por lo tanto, al multiplicarlos por los escalares `omega` y `1 - omega` y sumar las matrices resultantes, obtendrá una matriz que contiene $\{f^1\left(\omega\right), \cdots, f^k\left(\omega\right)\}$.

In [ ]:
def f_of_omega(omega):
    ### START CODE HERE ### (~ 1 line of code)
    f = prices_A * omega + prices_B * (1 - omega)
    ### END CODE HERE ###
    return f

def L_of_omega(omega):
    return 1/len(f_of_omega(omega)) * np.sum((f_of_omega(omega) - np.mean(f_of_omega(omega)))**2)

In [ ]:
print("L(omega = 0) =",L_of_omega(0))
print("L(omega = 0.2) =",L_of_omega(0.2))
print("L(omega = 0.8) =",L_of_omega(0.8))
print("L(omega = 1) =",L_of_omega(1))

**Salida esperada**
```
L(omega = 0) = 110.72
L(omega = 0.2) = 61.156807
L(omega = 0.8) = 11.212797
L(omega = 1) = 27.48
```

Analizando el resultado anterior, puedes notar que los valores de la función $\mathcal{L}$ disminuyen para $\omega$ aumentando de $0$ a $0.2$, luego a $0.8$, pero hay un aumento de la función $\mathcal{L}$ cuando $\omega = 1$. ¿Cuál será el $\omega$ que dará el valor mínimo de la función $\mathcal{L}$?

En este ejemplo simple $\mathcal{L}\left(\omega\right)$ es una función de una variable y el problema de encontrar su punto mínimo con cierta precisión es una tarea trivial. Solo necesitas calcular los valores de la función para cada $\omega = 0, 0.001, 0.002, \cdots , 1$ y encontrar el elemento mínimo de la matriz resultante.

La función `L_of_omega` no funcionará si pasa una matriz en lugar de un único valor de `omega` (no fue diseñada para eso). Es posible reescribirla de una manera que sea posible, pero en este caso no es necesario hacerlo ahora mismo: puede calcular los valores resultantes en el bucle, ya que no habrá tantos.

### Ejercicio 3

Evalúa la función `L_of_omega` para cada uno de los elementos del array `omega_array` y pasa el resultado al elemento correspondiente del array `L_array` con la función `.at[<index>].set(<value>)`.

*Nota*: se ha cargado `jax.numpy` en lugar del `NumPy` original. Hasta este momento, la funcionalidad `jax` no se ha utilizado realmente, pero se llamará en las celdas siguientes. Por lo tanto, no fue necesario cargar ambas versiones del paquete, y debes usar la función `.at[<index>].set(<value>)` para actualizar el array.

In [ ]:
# Parameter endpoint=True will allow ending point 1 to be included in the array.
# This is why it is better to take N = 1001, not N = 1000
N = 1001
omega_array = np.linspace(0, 1, N, endpoint=True)

# This is organised as a function only for grading purposes.
def L_of_omega_array(omega_array):
    N = len(omega_array)
    L_array = np.zeros(N)

    for i in range(N):
        ### START CODE HERE ### (~ 2 lines of code)
        L = L_of_omega(omega_array[i])
        L_array = L_array.at[i].set(L)
        ### END CODE HERE ###

    return L_array

L_array = L_of_omega_array(omega_array)

In [ ]:
print("L(omega = 0) =",L_array[0])
print("L(omega = 1) =",L_array[N-1])

**Salida esperada**

```
L(omega = 0) = 110.72
L(omega = 1) = 27.48
```

Ahora se puede encontrar un punto mínimo de la función $\mathcal{L}\left(\omega\right)$ con una función `NumPy` llamada `argmin()`. Como se tomaron $N = 1001$ puntos en el segmento $\left[0, 1\right]$, el resultado será preciso hasta tres decimales:

In [ ]:
i_opt = L_array.argmin()
omega_opt = omega_array[i_opt]
L_opt = L_array[i_opt]
print(f'omega_min = {omega_opt:.3f}\nL_of_omega_min = {L_opt:.7f}')

Este resultado significa que, según los datos históricos, se espera que $\omega = 0,702$ sea la opción más rentable para la distribución entre los proveedores A y B. Es razonable planificar que el $70,2\%$ del producto P sea suministrado por la Compañía A y el $29,8\%$ por la Compañía B.

Si desea mejorar la precisión, solo necesita aumentar la cantidad de puntos N. Este es un ejemplo muy simple de un modelo con un parámetro, lo que da como resultado la optimización de una función de una variable. Es computacionalmente barato evaluarlo en muchos puntos para encontrar el mínimo con cierta precisión. Pero en el aprendizaje automático, los modelos tienen cientos de parámetros; con un enfoque similar, necesitaría realizar millones de evaluaciones de la función objetivo. Esto no es posible en la mayoría de los casos, y ahí es donde entra en juego el cálculo con sus métodos y enfoques.

[comment]: En las próximas semanas de este curso, aprenderá a optimizar funciones multivariadas mediante la diferenciación. Pero por ahora, como estás en la curva de aprendizaje, evaluemos la derivada de la función $\mathcal{L}\left(\omega\right)$ en los puntos guardados en la matriz `omega_array` para verificar que en el punto mínimo la derivada es en realidad la más cercana a cero.

### Ejercicio 4

Para cada $\omega$ en el `omega_array` calcula $\frac{d\mathcal{L}}{d\omega}$ usando la función `grad()` de la biblioteca `JAX` (**La abreviatura** `grad` **proviene de**  *gradiente*, **otra palabra de la literatura científica y técnica para "derivada"**). Recuerda que necesitas pasar la función que quieres diferenciar (aquí $\mathcal{L}\left(\omega\right)$) como argumento de la función `grad()` y luego evaluar la derivada para el elemento correspondiente del `omega_array`. Luego pasa el resultado al elemento correspondiente del array `dLdOmega_array` con la función `.at[<index>].set(<value>)`.

In [ ]:
# This is organised as a function only for grading purposes.
def dLdOmega_of_omega_array(omega_array):
    N = len(omega_array)
    dLdOmega_array = np.zeros(N)

    for i in range(N):
        ### START CODE HERE ### (~ 2 lines of code)
        dLdOmega = grad(L_of_omega)(omega_array[i])
        dLdOmega_array = dLdOmega_array.at[i].set(dLdOmega)
        ### END CODE HERE ###

    return dLdOmega_array

dLdOmega_array = dLdOmega_of_omega_array(omega_array)

In [ ]:
print("dLdOmega(omega = 0) =",dLdOmega_array[0])
print("dLdOmega(omega = 1) =",dLdOmega_array[N-1])

**Salida esperada**
```
dLdOmega(omega = 0) = -288.96
dLdOmega(omega = 1) = 122.47999
```

Ahora, para encontrar el valor más cercano de la derivada a $0$, tome los valores absolutos $\left|\frac{d\mathcal{L}}{d\omega}\right|$ para cada omega y encuentre el mínimo de ellos.

In [ ]:
i_opt_2 = np.abs(dLdOmega_array).argmin()
omega_opt_2 = omega_array[i_opt_2]
dLdOmega_opt_2 = dLdOmega_array[i_opt_2]
print(f'omega_min = {omega_opt_2:.3f}\ndLdOmega_min = {dLdOmega_opt_2:.7f}')

El resultado es el mismo: $\omega = 0.702$. Grafiquemos $\mathcal{L}\left(\omega\right)$ y $\frac{d\mathcal{L}}{d\omega}$ para visualizar sus gráficas, el punto mínimo de la función $\mathcal{L}\left(\omega\right)$ y el punto donde su derivada está alrededor de $0$:

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
# Setting the axes at the origin.
ax.spines['left'].set_position('zero')
ax.spines['bottom'].set_position('zero')
ax.spines['right'].set_color('none')
ax.spines['top'].set_color('none')
ax.xaxis.set_ticks_position('bottom')
ax.yaxis.set_ticks_position('left')

plt.plot(omega_array,  L_array, "black", label = "$\mathcal{L}\\left(\omega\\right)$")
plt.plot(omega_array,  dLdOmega_array, "orange", label = "$\mathcal{L}\'\\left(\omega\\right)$")
plt.plot([omega_opt, omega_opt_2], [L_opt,dLdOmega_opt_2], 'ro', markersize=3)

plt.legend()

plt.show()

¡Felicitaciones! ¡Has terminado la tarea! Este ejemplo ilustra cómo pueden aparecer los problemas de optimización en la vida real y te da la oportunidad de explorar el caso simple de minimizar una función con una variable. ¡Ahora es el momento de aprender sobre la optimización de funciones multivariadas!